# PROMAT 2025
### March 17-20, 2025
McCormick Place

Contains: Company Name, City, State, Phone, Website, Booth, Booth Size, Hall

In [1]:
import pandas as pd
import requests
import re
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time
from selenium.common.exceptions import NoSuchElementException, TimeoutException  # Import TimeoutException

In [2]:
# Specify the path to your ChromeDriver
chrome_driver_path = "/Users/graceennema/Downloads/chromedriver-mac-x64/chromedriver"  # Replace with your path

# Create a service object for ChromeDriver (newer versions require this)
service = Service(executable_path=chrome_driver_path)

chrome_options = Options()
#chrome_options.add_argument("start-maximized")
chrome_options.add_argument("--headless")  # Run in headless mode

driver = webdriver.Chrome(service=service, options=chrome_options)

# Open the website you want to scrape
driver.get("https://pm2025.mapyourshow.com/8_0/exhview/index.cfm?hallID=A")  # Replace with the actual URL
#https://pm2025.mapyourshow.com/8_0/exhview/index.cfm?hallID=A  --> Lakeside center
#https://pm2025.mapyourshow.com/8_0/exhview/index.cfm?hallID=B --> North Building
#https://pm2025.mapyourshow.com/8_0/exhview/index.cfm?hallID=S --> South building

# Wait for the exhibitor section to load
WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "matchingExhibitorsTable")))

# Try scrolling to ensure the content is fully loaded
actions = ActionChains(driver)
actions.move_to_element(driver.find_element(By.ID, "matchingExhibitorsTable")).perform()

time.sleep(5)
# Wait for the page to load and extract the page source
html = driver.page_source

# Now you can use BeautifulSoup to parse the page
soup = BeautifulSoup(html, "html.parser")


In [3]:
data =[]
# Print out the table content again to see if it's populated
table = soup.find("table", id="matchingExhibitorsTable")
# Loop through each row in the table
for row in table.find_all('tr'):
    # Find the link containing the booth data and other details
    booth_link = row.find('a', class_='js-exhShowroomPanelLink')
    
    if booth_link:     
        # Extract the company name and booth info text, skipping the booth number part
        company_and_booth_info = booth_link.get_text(separator='|').split('|')
        
        # Clean and extract the company name and booth size info
        company_and_booth_info = [info.strip() for info in company_and_booth_info if info.strip()]

        # Company name is typically in the first part and booth size/type in the second
        if len(company_and_booth_info) >= 2:
            booth_number = company_and_booth_info[0].split('\n')[0].strip()  # Booth number (clean)
            company_name = company_and_booth_info[0].split('\n')[-1].strip()  # Company name (clean)
            if company_name.startswith("-"):
                company_name = company_name[1:].strip()

            # Split the booth details part into size and type
            booth_details_parts = company_and_booth_info[1].split('-')  # Split by the hyphen that separates size and type
            booth_size = booth_details_parts[0].strip()  # Booth size (e.g., 10' x 10')
            booth_type = booth_details_parts[1].strip() if len(booth_details_parts) > 1 else ""  # Booth type (e.g., Perimeter Booth)
        
        hall = booth_link.get('data-hallid')
        hall2 = "Lakeside center"
        
        # Construct the full booth URL and navigate to it
        booth_url = f"https://pm2025.mapyourshow.com/8_0/exhview/index.cfm?hallID=A&selectedBooth={booth_number}"  # Construct the URL based on booth number
        driver.get(booth_url)
        
        print(f"Successfully loaded booth page for {booth_number}")

        # Wait for the booth page to load and scrape additional information
        try:
            # Adjust the waiting condition to find the elements on the booth page
            WebDriverWait(driver, 20).until(
                EC.presence_of_element_located((By.CLASS_NAME, 'mys-exh-contact-info'))  # Change this class name as needed
            )
            
            try:
                # Extract contact info
                contact_info = driver.find_element(By.CLASS_NAME, 'mys-exh-contact-info').get_attribute('innerHTML')

                # Use BeautifulSoup to parse the contact info
                soup_contact = BeautifulSoup(contact_info, 'html.parser')

                # Extract the address, city/state/zip, country, and phone
                contact_lines = [line.strip() for line in soup_contact.stripped_strings]

                address = contact_lines[0] if len(contact_lines) > 0 else ''
                city_state_zip = contact_lines[1] if len(contact_lines) > 1 else ''
                country = contact_lines[2] if len(contact_lines) > 2 else ''

                        # Split city, state, zip if available
                city, state, zip_code = '', '', ''
                if city_state_zip:
                    parts = city_state_zip.split(',')
                    if len(parts) > 1:
                        city = parts[0].strip()
                        state_zip = parts[1].strip().split(' ')
                        state = state_zip[0] if len(state_zip) > 0 else ''
                        zip_code = state_zip[1] if len(state_zip) > 1 else ''
                
                
                
                # Find phone number
                phone_line = next((line for line in contact_lines if line.startswith('P:')), '')
                phone = phone_line.replace('P: ', '').strip() if phone_line else ''  # Remove "P: " prefix

                # Extract the website URL
                website_link = soup_contact.find('a', class_='js-websiteLink')
                website = website_link.get('href') if website_link else ''

                data.append({
                    'Company Name': company_name,
                    'City': city,
                    'State': state,
                    'Country': country,
                    'Phone': phone,
                    'Website': website,
                    'Booth Number': booth_number,
                    'Booth Size': booth_size,
                    'Booth Type': booth_type,
                    'Hall': hall,
                    'Hall Full': hall2,
                    'Address': address
                })

            except Exception as e:
                print(f"Error extracting data from booth page for {booth_number}")

        except Exception as e:
            print(f"Error waiting for booth page to load for {booth_number}")
            data.append({
                    'Company Name': company_name,
                    'Booth Number': booth_number,
                    'Booth Size': booth_size,
                    'Booth Type': booth_type,
                    'Hall': hall,
                    'Hall Full': hall2
                })

        driver.back()
        

Successfully loaded booth page for E10000
Error waiting for booth page to load for E10000
Successfully loaded booth page for E10621
Successfully loaded booth page for E10622
Successfully loaded booth page for E10623
Successfully loaded booth page for E10625
Successfully loaded booth page for E10626
Successfully loaded booth page for E10627
Successfully loaded booth page for E10628
Successfully loaded booth page for E10629
Successfully loaded booth page for E10630
Successfully loaded booth page for E10632
Successfully loaded booth page for E10634
Successfully loaded booth page for E10635
Successfully loaded booth page for E10637
Successfully loaded booth page for E10638
Successfully loaded booth page for E10639
Successfully loaded booth page for E10640
Successfully loaded booth page for E10643
Successfully loaded booth page for E10646
Successfully loaded booth page for E10648
Successfully loaded booth page for E10651
Successfully loaded booth page for E10653
Successfully loaded booth pa

Successfully loaded booth page for E12327
Successfully loaded booth page for E12329
Successfully loaded booth page for E12330
Successfully loaded booth page for E12331
Successfully loaded booth page for E12334
Successfully loaded booth page for E12336
Successfully loaded booth page for E12337
Successfully loaded booth page for E12338
Successfully loaded booth page for E12339
Successfully loaded booth page for E12341
Successfully loaded booth page for E12343
Error waiting for booth page to load for E12343
Successfully loaded booth page for E12345
Successfully loaded booth page for E12346
Successfully loaded booth page for E12348
Successfully loaded booth page for E12351
Successfully loaded booth page for E12353
Successfully loaded booth page for E12354
Successfully loaded booth page for E12355
Successfully loaded booth page for E12412
Successfully loaded booth page for E12413
Successfully loaded booth page for E12427
Successfully loaded booth page for E12428
Successfully loaded booth pa

Successfully loaded booth page for E13331
Error waiting for booth page to load for E13331
Successfully loaded booth page for E13332
Error waiting for booth page to load for E13332
Successfully loaded booth page for E13333
Error waiting for booth page to load for E13333
Successfully loaded booth page for E13335
Error waiting for booth page to load for E13335
Successfully loaded booth page for E13336
Error waiting for booth page to load for E13336
Successfully loaded booth page for E13337
Error waiting for booth page to load for E13337
Successfully loaded booth page for E13338
Error waiting for booth page to load for E13338
Successfully loaded booth page for E13339
Error waiting for booth page to load for E13339
Successfully loaded booth page for E13340
Error waiting for booth page to load for E13340
Successfully loaded booth page for E13341


In [4]:
df = pd.DataFrame(data)
df
df.to_excel('PROMAT_A_2025.xlsx', index=False)

In [5]:
# Open the website you want to scrape
driver.get("https://pm2025.mapyourshow.com/8_0/exhview/index.cfm?hallID=B")  # Replace with the actual URL
#https://pm2025.mapyourshow.com/8_0/exhview/index.cfm?hallID=A  --> Lakeside center
#https://pm2025.mapyourshow.com/8_0/exhview/index.cfm?hallID=B --> North Building
#https://pm2025.mapyourshow.com/8_0/exhview/index.cfm?hallID=S --> South building

# Wait for the exhibitor section to load
WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "matchingExhibitorsTable")))

# Try scrolling to ensure the content is fully loaded
actions = ActionChains(driver)
actions.move_to_element(driver.find_element(By.ID, "matchingExhibitorsTable")).perform()

time.sleep(5)
# Wait for the page to load and extract the page source
html = driver.page_source

# Now you can use BeautifulSoup to parse the page
soup = BeautifulSoup(html, "html.parser")


In [6]:
#data =[]
# Print out the table content again to see if it's populated
table = soup.find("table", id="matchingExhibitorsTable")
# Loop through each row in the table
for row in table.find_all('tr'):
    # Find the link containing the booth data and other details
    booth_link = row.find('a', class_='js-exhShowroomPanelLink')
    
    if booth_link:     
        # Extract the company name and booth info text, skipping the booth number part
        company_and_booth_info = booth_link.get_text(separator='|').split('|')
        
        # Clean and extract the company name and booth size info
        company_and_booth_info = [info.strip() for info in company_and_booth_info if info.strip()]

        # Company name is typically in the first part and booth size/type in the second
        if len(company_and_booth_info) >= 2:
            booth_number = company_and_booth_info[0].split('\n')[0].strip()  # Booth number (clean)
            company_name = company_and_booth_info[0].split('\n')[-1].strip()  # Company name (clean)
            if company_name.startswith("-"):
                company_name = company_name[1:].strip()

            # Split the booth details part into size and type
            booth_details_parts = company_and_booth_info[1].split('-')  # Split by the hyphen that separates size and type
            booth_size = booth_details_parts[0].strip()  # Booth size (e.g., 10' x 10')
            booth_type = booth_details_parts[1].strip() if len(booth_details_parts) > 1 else ""  # Booth type (e.g., Perimeter Booth)
        
        hall = booth_link.get('data-hallid')
        hall2 = "North Building"
        
        # Construct the full booth URL and navigate to it
        booth_url = f"https://pm2025.mapyourshow.com/8_0/exhview/index.cfm?hallID=B&selectedBooth={booth_number}"  # Construct the URL based on booth number
        driver.get(booth_url)
        
        print(f"Successfully loaded booth page for {booth_number}")

        # Wait for the booth page to load and scrape additional information
        try:
            # Adjust the waiting condition to find the elements on the booth page
            WebDriverWait(driver, 20).until(
                EC.presence_of_element_located((By.CLASS_NAME, 'mys-exh-contact-info'))  # Change this class name as needed
            )
            
            try:
                # Extract contact info
                contact_info = driver.find_element(By.CLASS_NAME, 'mys-exh-contact-info').get_attribute('innerHTML')

                # Use BeautifulSoup to parse the contact info
                soup_contact = BeautifulSoup(contact_info, 'html.parser')

                # Extract the address, city/state/zip, country, and phone
                contact_lines = [line.strip() for line in soup_contact.stripped_strings]

                address = contact_lines[0] if len(contact_lines) > 0 else ''
                city_state_zip = contact_lines[1] if len(contact_lines) > 1 else ''
                country = contact_lines[2] if len(contact_lines) > 2 else ''

                        # Split city, state, zip if available
                city, state, zip_code = '', '', ''
                if city_state_zip:
                    parts = city_state_zip.split(',')
                    if len(parts) > 1:
                        city = parts[0].strip()
                        state_zip = parts[1].strip().split(' ')
                        state = state_zip[0] if len(state_zip) > 0 else ''
                        zip_code = state_zip[1] if len(state_zip) > 1 else ''
                
                
                
                # Find phone number
                phone_line = next((line for line in contact_lines if line.startswith('P:')), '')
                phone = phone_line.replace('P: ', '').strip() if phone_line else ''  # Remove "P: " prefix

                # Extract the website URL
                website_link = soup_contact.find('a', class_='js-websiteLink')
                website = website_link.get('href') if website_link else ''

                data.append({
                    'Company Name': company_name,
                    'City': city,
                    'State': state,
                    'Country': country,
                    'Phone': phone,
                    'Website': website,
                    'Booth Number': booth_number,
                    'Booth Size': booth_size,
                    'Booth Type': booth_type,
                    'Hall': hall,
                    'Hall Full': hall2,
                    'Address': address
                })

            except Exception as e:
                print(f"Error extracting data from booth page for {booth_number}")

        except Exception as e:
            print(f"Error waiting for booth page to load for {booth_number}")
            data.append({
                    'Company Name': company_name,
                    'Booth Number': booth_number,
                    'Booth Size': booth_size,
                    'Booth Type': booth_type,
                    'Hall': hall,
                    'Hall Full': hall2
                })

        driver.back()
        

Successfully loaded booth page for N6001
Successfully loaded booth page for N6002
Successfully loaded booth page for N6003
Successfully loaded booth page for N6004
Successfully loaded booth page for N6006
Successfully loaded booth page for N6007
Successfully loaded booth page for N6010
Successfully loaded booth page for N6011
Successfully loaded booth page for N6012
Successfully loaded booth page for N6015
Successfully loaded booth page for N6018
Successfully loaded booth page for N6021
Successfully loaded booth page for N6022
Successfully loaded booth page for N6027
Successfully loaded booth page for N6028
Successfully loaded booth page for N6029
Successfully loaded booth page for N6031
Successfully loaded booth page for N6033
Successfully loaded booth page for N6035
Successfully loaded booth page for N6036
Successfully loaded booth page for N6039
Successfully loaded booth page for N6040
Successfully loaded booth page for N6041
Successfully loaded booth page for N6042
Successfully loa

Successfully loaded booth page for N8519
Successfully loaded booth page for N8522
Successfully loaded booth page for N8524
Successfully loaded booth page for N8527
Successfully loaded booth page for N8530
Successfully loaded booth page for N8534
Successfully loaded booth page for N8535
Successfully loaded booth page for N8536
Successfully loaded booth page for N8539
Successfully loaded booth page for N8541
Successfully loaded booth page for N8544
Successfully loaded booth page for N8545
Successfully loaded booth page for N8546
Successfully loaded booth page for N8550
Successfully loaded booth page for N8553
Successfully loaded booth page for N8556
Successfully loaded booth page for N8559
Successfully loaded booth page for N8563
Error waiting for booth page to load for N8563
Successfully loaded booth page for N8563
Error waiting for booth page to load for N8563


In [7]:
df = pd.DataFrame(data)
df
df.to_excel('PROMAT_B_2025.xlsx', index=False)

In [8]:
# Open the website you want to scrape
driver.get("https://pm2025.mapyourshow.com/8_0/exhview/index.cfm?hallID=S")  # Replace with the actual URL
#https://pm2025.mapyourshow.com/8_0/exhview/index.cfm?hallID=A  --> Lakeside center
#https://pm2025.mapyourshow.com/8_0/exhview/index.cfm?hallID=B --> North Building
#https://pm2025.mapyourshow.com/8_0/exhview/index.cfm?hallID=S --> South building

# Wait for the exhibitor section to load
WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "matchingExhibitorsTable")))

# Try scrolling to ensure the content is fully loaded
actions = ActionChains(driver)
actions.move_to_element(driver.find_element(By.ID, "matchingExhibitorsTable")).perform()

time.sleep(5)
# Wait for the page to load and extract the page source
html = driver.page_source

# Now you can use BeautifulSoup to parse the page
soup = BeautifulSoup(html, "html.parser")

In [9]:
#data =[]
# Print out the table content again to see if it's populated
table = soup.find("table", id="matchingExhibitorsTable")
# Loop through each row in the table
for row in table.find_all('tr'):
    # Find the link containing the booth data and other details
    booth_link = row.find('a', class_='js-exhShowroomPanelLink')
    
    if booth_link:     
        # Extract the company name and booth info text, skipping the booth number part
        company_and_booth_info = booth_link.get_text(separator='|').split('|')
        
        # Clean and extract the company name and booth size info
        company_and_booth_info = [info.strip() for info in company_and_booth_info if info.strip()]

        # Company name is typically in the first part and booth size/type in the second
        if len(company_and_booth_info) >= 2:
            booth_number = company_and_booth_info[0].split('\n')[0].strip()  # Booth number (clean)
            company_name = company_and_booth_info[0].split('\n')[-1].strip()  # Company name (clean)
            if company_name.startswith("-"):
                company_name = company_name[1:].strip()

            # Split the booth details part into size and type
            booth_details_parts = company_and_booth_info[1].split('-')  # Split by the hyphen that separates size and type
            booth_size = booth_details_parts[0].strip()  # Booth size (e.g., 10' x 10')
            booth_type = booth_details_parts[1].strip() if len(booth_details_parts) > 1 else ""  # Booth type (e.g., Perimeter Booth)
        
        hall = booth_link.get('data-hallid')
        hall2 = "North Building"
        
        # Construct the full booth URL and navigate to it
        booth_url = f"https://pm2025.mapyourshow.com/8_0/exhview/index.cfm?hallID=S&selectedBooth={booth_number}"  # Construct the URL based on booth number
        driver.get(booth_url)
        
        print(f"Successfully loaded booth page for {booth_number}")

        # Wait for the booth page to load and scrape additional information
        try:
            # Adjust the waiting condition to find the elements on the booth page
            WebDriverWait(driver, 20).until(
                EC.presence_of_element_located((By.CLASS_NAME, 'mys-exh-contact-info'))  # Change this class name as needed
            )
            
            try:
                # Extract contact info
                contact_info = driver.find_element(By.CLASS_NAME, 'mys-exh-contact-info').get_attribute('innerHTML')

                # Use BeautifulSoup to parse the contact info
                soup_contact = BeautifulSoup(contact_info, 'html.parser')

                # Extract the address, city/state/zip, country, and phone
                contact_lines = [line.strip() for line in soup_contact.stripped_strings]

                address = contact_lines[0] if len(contact_lines) > 0 else ''
                city_state_zip = contact_lines[1] if len(contact_lines) > 1 else ''
                country = contact_lines[2] if len(contact_lines) > 2 else ''

                        # Split city, state, zip if available
                city, state, zip_code = '', '', ''
                if city_state_zip:
                    parts = city_state_zip.split(',')
                    if len(parts) > 1:
                        city = parts[0].strip()
                        state_zip = parts[1].strip().split(' ')
                        state = state_zip[0] if len(state_zip) > 0 else ''
                        zip_code = state_zip[1] if len(state_zip) > 1 else ''
                
                
                
                # Find phone number
                phone_line = next((line for line in contact_lines if line.startswith('P:')), '')
                phone = phone_line.replace('P: ', '').strip() if phone_line else '' 

                # Extract the website URL
                website_link = soup_contact.find('a', class_='js-websiteLink')
                website = website_link.get('href') if website_link else ''

                data.append({
                    'Company Name': company_name,
                    'City': city,
                    'State': state,
                    'Country': country,
                    'Phone': phone,
                    'Website': website,
                    'Booth Number': booth_number,
                    'Booth Size': booth_size,
                    'Booth Type': booth_type,
                    'Hall': hall,
                    'Hall Full': hall2,
                    'Address': address
                })

            except Exception as e:
                print(f"Error extracting data from booth page for {booth_number}")

        except Exception as e:
            print(f"Error waiting for booth page to load for {booth_number}")
            data.append({
                    'Company Name': company_name,
                    'Booth Number': booth_number,
                    'Booth Size': booth_size,
                    'Booth Type': booth_type,
                    'Hall': hall,
                    'Hall Full': hall2
                })

        driver.back()
        

Successfully loaded booth page for S1003
Successfully loaded booth page for S1013
Successfully loaded booth page for S1018
Successfully loaded booth page for S1020
Successfully loaded booth page for S1021
Successfully loaded booth page for S1025
Successfully loaded booth page for S103
Successfully loaded booth page for S1031
Successfully loaded booth page for S1037
Successfully loaded booth page for S1041
Successfully loaded booth page for S1050
Successfully loaded booth page for S1052
Successfully loaded booth page for S1056
Successfully loaded booth page for S106
Successfully loaded booth page for S1063
Successfully loaded booth page for S1068
Successfully loaded booth page for S1070
Successfully loaded booth page for S1073
Successfully loaded booth page for S1075
Successfully loaded booth page for S1077
Successfully loaded booth page for S108
Successfully loaded booth page for S110
Successfully loaded booth page for S1118
Successfully loaded booth page for S112
Successfully loaded b

Successfully loaded booth page for S2275
Successfully loaded booth page for S2278
Successfully loaded booth page for S2280
Successfully loaded booth page for S2282
Successfully loaded booth page for S2284
Successfully loaded booth page for S2303
Successfully loaded booth page for S231
Successfully loaded booth page for S2312
Successfully loaded booth page for S2315
Successfully loaded booth page for S2316
Successfully loaded booth page for S2318
Error waiting for booth page to load for S2318
Successfully loaded booth page for S2318
Error waiting for booth page to load for S2318
Successfully loaded booth page for S2323
Successfully loaded booth page for S2326
Error waiting for booth page to load for S2326
Successfully loaded booth page for S233
Successfully loaded booth page for S2331
Successfully loaded booth page for S2337
Successfully loaded booth page for S234
Successfully loaded booth page for S2347
Successfully loaded booth page for S2352
Successfully loaded booth page for S2363
S

Successfully loaded booth page for S3831
Error waiting for booth page to load for S3831
Successfully loaded booth page for S3837
Successfully loaded booth page for S3841
Successfully loaded booth page for S3847
Successfully loaded booth page for S3853
Successfully loaded booth page for S3859
Successfully loaded booth page for S3868
Successfully loaded booth page for S3870
Successfully loaded booth page for S3872
Successfully loaded booth page for S3875
Error waiting for booth page to load for S3875
Successfully loaded booth page for S3875
Error waiting for booth page to load for S3875
Successfully loaded booth page for S3875
Error waiting for booth page to load for S3875
Successfully loaded booth page for S3879
Successfully loaded booth page for S4018
Successfully loaded booth page for S4020
Successfully loaded booth page for S403
Successfully loaded booth page for S4068
Error waiting for booth page to load for S4068
Successfully loaded booth page for S4068
Error waiting for booth page

In [10]:
df = pd.DataFrame(data)
df

,Company Name,Booth Number,Booth Size,Booth Type,Hall,Hall Full,City,State,Country,Phone,Website,Address
0,E10000,E10000,30' x 60',Island Booth,A,Lakeside center,NaN,NaN,NaN,NaN,NaN,NaN
1,"L-S Industries, Inc.",E10621,10' x 10',Perimeter Booth,A,Lakeside center,Knoxville,TN,United States,1-800-825-2561,https://www.l-si.com,8902 Valgro Road
2,Longhe Intelligent Equipment Manufacturing Co....,E10622,10' x 10',Perimeter Booth,A,Lakeside center,Longyan,Fujian,United States of America,+86-18805976610,https://www.longheglobal.com/,"No.9, Yuanyuan Road, High-tech Zone"
3,Munbyn,E10623,20' x 10',Perimeter Booth,A,Lakeside center,Guangzhou,Guangdong,China,+86 18027460382,http://www.munbyn.com,"501-2, Building C10, No. 63 Xizeng Road, Liwan..."
4,Cresa,E10625,10' x 10',Perimeter Booth,A,Lakeside center,Atlanta,Georgia,United States of America,404.257.8866,https://www.cresa.com/,"100 Galleria Parkway, Suite 200"
...,...,...,...,...,...,...,...,...,...,...,...,...
1135,Impact Recovery,S668,25' x 20',Island Booth,S,North Building,San Antonio,TX,United States of America,(210) 736-4477,https://www.impactrecovery.com/,4955 Stout Dr.
1136,Micropower Group US,S671,30' x 20',Island Booth,S,North Building,Troy,OH,United States,937-606-2793,http://www.ecotecbatcharger.com,150 Marybill Dr. S.
1137,Tri-Boro Rack & Storage Products,S868,30' x 20',Island Booth,S,North Building,NaN,NaN,NaN,NaN,NaN,NaN
1138,Tri-Boro Shelving & Partition Corp,S868,30' x 20',Island Booth,S,North Building,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
driver.quit()

In [12]:
df.to_excel('PROMAT_S_2025.xlsx', index=False)